# Use Gensim to vectorize features

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import numexpr as ne
import mskcc_functions as ski
import scipy.stats as scs
import feature_engineering as fe
import xgboost as xgb

from pprint import pprint
from matplotlib  import cm
from collections import Counter
from importlib import reload
from gensim import corpora, matutils, models, similarities
from nltk import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import scale, normalize, robust_scale
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA

sns.set_context("paper")
%matplotlib inline

In [2]:
class_train = pd.read_csv('./data/training_variants')
text_train = pd.read_csv("./data/training_text", sep="\|\|", engine='python',
                         header=None, skiprows=1, names=["ID","Text"])

feature_matrix1 = pd.read_csv('./data/feature_matrix_pass1.csv')
print("Feature Matrix1 Shape Before: ", feature_matrix1.shape)

total_frequency = feature_matrix1.sum(axis=0)
zero_frequency_index = list(np.argwhere(total_frequency == 0).ravel())
nonzero_frequency_index = [ind for ind in range(feature_matrix1.shape[1]) if ind not in zero_frequency_index]
feature_matrix1 = feature_matrix1.iloc[:, nonzero_frequency_index]
print("Feature Matrix1 Shape After: ", feature_matrix1.shape)

feature_words = list(feature_matrix1.columns)

Feature Matrix1 Shape Before:  (3321, 22190)
Feature Matrix1 Shape After:  (3321, 22156)


In [3]:
additional_removal = ['of', 'out', 'one', 'the', 'and', 'from', 'that', 'this', 'these', 'no',
                     'those', 'he', 'she', 'me', 'you', 'her', 'by','we', 'is', 'to', 'at', 
                      'into', 'onto', 'on', 'out', 'can', 'should', 'must', 'would', 'will',
                      'as', 'between', 'within','among','which', 'where', 'when','how','mm',
                     'na', 'many', 'much', 'min', 'sec', 'hr', 'go', 'via']
stop_words = set(stopwords.words('english'))

In [4]:
print('# of words before removal: %d' % len(feature_words))
feature_words = [word for word in feature_words \
                 if word not in stop_words \
                 if word not in additional_removal]
print('# of words after removal: %d' % len(feature_words))

# of words before removal: 22156
# of words after removal: 22089


In [5]:
# redundancy from plural forms to be removed and taken care of.
# This requires some fiddling since the word still has to be found
# feature_words = [re.sub(r's$', '', word) for word in feature_words]

In [6]:
%%time
documents = []
for i in range(10):
    document = text_train['Text'].iloc[i] + ''
    documents.append(document)

CPU times: user 289 µs, sys: 0 ns, total: 289 µs
Wall time: 295 µs


In [7]:
%%time
textome = [[word for word in document.lower().split() if word in feature_words] for document in documents]

CPU times: user 13.4 s, sys: 1.08 ms, total: 13.4 s
Wall time: 13.4 s


This would take too much time (>1hr) to process, so use subset of the feature_words space as below

<b>With narrower feature space</b>

In [8]:
%%time
whole_text = ''
for i in range(len(text_train)):
    text = text_train.loc[i, 'Text'] + ''
    whole_text += text

# Remove irrelevant characters by replacing them with whitespace
whole_text_white = fe.replace_with_whitespace(whole_text, hyphens='on')

# Tokenize the whole_text_white
tokens_white = word_tokenize(whole_text_white)

In [9]:
%%time
# Get race words
race_words = set(fe.get_race_words(tokens_white))

#Get drug words
drug_words = set(fe.get_drug_words(tokens_white))

#Get tissue type words (organ, tumor type, and cell type), combine the#
tissue_type_words = set(fe.get_tissue_type_words(tokens_white))

CPU times: user 1min 51s, sys: 999 µs, total: 1min 51s
Wall time: 1min 51s


In [10]:
# Get number of words obtained from each
print('# of race words: %d' % len(race_words))
print('# of drug words: %d' % len(drug_words))
print('# of tissue type words: %d' % len(tissue_type_words))

# of race words: 10
# of drug words: 202
# of tissue type words: 467


In [11]:
feature_words2 = list(race_words) + list(drug_words) + list(tissue_type_words)
feature_words2 = set(feature_words2)

feature_words2 = [re.sub(r'^[\W0-9]+', '', word) for word in feature_words2]
feature_words2 = [word for word in feature_words2 if word not in additional_removal]
feature_words2 = sorted(list(set(feature_words2)))
print(len(feature_words2))

662


In [12]:
# save the 1st pass feature names
pd.Series(feature_words2).to_csv('./data/features_pass1_subset1.csv', index=False)

In [20]:
mskcc_dictionary1 = corpora.Dictionary([feature_words2])
mskcc_dictionary1.save('/tmp/mskcc_dictionary1.dict')

In [21]:
print(mskcc_dictionary1.token2id)

{'aadenocarcinoma': 0, 'abagovomab': 1, 'acanthoma': 2, 'acthoma': 3, 'adamantinoma': 4, 'adenenocarcinoma': 5, 'adenoacanthoma': 6, 'adenocanthoma': 7, 'adenocarcinoma': 8, 'adenocarcioma': 9, 'adenocarinoma': 10, 'adenocrcinoma': 11, 'adenolymphoma': 12, 'adenoma': 13, 'adenoma–carcinoma': 14, 'adenomyoepithelialoma': 15, 'adenomyoepithelioma': 16, 'adenomyoepithioma': 17, 'adeocarcinoma': 18, 'adipocyte': 19, 'administrationcrizotinib': 20, 'adnocarcinoma': 21, 'adrenal': 22, 'afatanib': 23, 'afatinib': 24, 'african': 25, 'agerminoma': 26, 'alectinib': 27, 'alemtuzumab': 28, 'alk+lymphoma': 29, 'alkoma': 30, 'altiratinib': 31, 'ampullarycarcinoma': 32, 'amuvatinib': 33, 'andlymphocyte': 34, 'angiofibroma': 35, 'angioleiomyoma': 36, 'angioma': 37, 'angiomyofibroblastoma': 38, 'angiomyolipoma': 39, 'angiomyxoma': 40, 'angiosarcoma': 41, 'anoma': 42, 'antihepatocyte': 43, 'antimelanoma': 44, 'antimyeloma': 45, 'anti–glioblastoma': 46, 'arcinoma': 47, 'arms—erlotinib': 48, 'aroma': 49, 

## Start over with saved files

In [20]:
feature_words2= pd.read_csv('./data/features_pass1_subset1.csv', header=None)
feature_words2 = list(feature_words2[0])

In [22]:
dict1 = corpora.Dictionary([feature_words2])

In [23]:
%%time
documents = []
tokenized_documents = []
for i in range(len(text_train)):
    doc= text_train['Text'].iloc[i]
    tokenized_doc = word_tokenize(fe.replace_with_whitespace(doc, hyphens='on'))
    
    documents.append(doc)
    tokenized_documents.append(tokenized_doc)

print(len(documents))     # double checking length
print(len(tokenized_documents))     # double checking length

3321
3321
CPU times: user 1min 55s, sys: 489 ms, total: 1min 55s
Wall time: 1min 55s


In [29]:
%time
corpus = [dict1.doc2bow(token) for token in tokenized_documents]
tfidf = models.TfidfModel(corpus)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [36]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf[:5]:
    print(doc)

[(68, 0.10180953731779364), (69, 0.28417097593268426), (102, 0.30423889726812786), (125, 0.5501255110547993), (126, 0.08571365011797719), (261, 0.18349439254579827), (308, 0.21121164590425284), (414, 0.5200431794352696), (545, 0.3970012597830717)]
[(8, 0.13839522847728142), (69, 0.03125203608407436), (82, 0.08968356109962472), (126, 0.028279377345904216), (308, 0.06968474480032831), (335, 0.0363244362091049), (341, 0.9802441363339809), (348, 0.04208912438416052), (369, 0.029899754762902034), (534, 0.03616243622163127)]
[(8, 0.13839522847728142), (69, 0.03125203608407436), (82, 0.08968356109962472), (126, 0.028279377345904216), (308, 0.06968474480032831), (335, 0.0363244362091049), (341, 0.9802441363339809), (348, 0.04208912438416052), (369, 0.029899754762902034), (534, 0.03616243622163127)]
[(65, 0.4018172964218958), (327, 0.9026776097619474), (348, 0.12554575775806914), (369, 0.08918663487097019)]
[(101, 0.6071315660697193), (308, 0.1844343787916951), (341, 0.2937066618238871), (348, 

In [40]:
feature_matrix = matutils.corpus2dense(corpus_tfidf, num_terms=len(feature_words2), num_docs=len(corpus_tfidf)).T

In [42]:
pd.DataFrame(feature_matrix).to_csv('feature_matrix_pass1_subset1.csv', index=False)

<b>Try RFC</b>

In [53]:
# Import Data and extract gene and mutation type info
# Get Gene feature from 'train_variants' data
X_gene = np.array(class_train.Gene)
X_gene_int = LabelEncoder().fit_transform(X_gene.ravel()).reshape(-1, 1)
X_gene_bin = OneHotEncoder().fit_transform(X_gene_int).toarray()
gene_table = pd.DataFrame(X_gene_bin)

# Get Mutation Type from 'train_variants' data
mut_type = ski.convert_mutation_type(class_train)
X_mtype = np.array(mut_type['mutation_type'])
X_mtype_int = LabelEncoder().fit_transform(X_mtype.ravel()).reshape(-1, 1)
X_mtype_bin = OneHotEncoder().fit_transform(X_mtype_int).toarray()
mtype_table = pd.DataFrame(X_mtype_bin)

feat_mat = pd.DataFrame(feature_matrix)

In [55]:
features = pd.concat([gene_table, mtype_table, feat_mat], axis=1)
X = np.array(features).astype(float)
y = np.array(class_train.Class).astype(int).ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [56]:
%%time
# Run RFC on the data
rfc = RandomForestClassifier(n_estimators=100, max_depth=100, n_jobs=4)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
y_proba = rfc.predict_proba(X_test)
print(accuracy_score(y_test, y_pred))
print(log_loss(y_test, y_proba))

0.681362725451
1.35105150702
CPU times: user 1.21 s, sys: 16.1 ms, total: 1.23 s
Wall time: 569 ms


<b>XGBoost</b>

Play with max_depth

In [65]:
%%time
# Adjust the class labels so it starts with 0
y_adj = y-1

# Split into test and train
x1, x2, y1, y2 = train_test_split(X, y_adj, test_size=0.15)

depths =[10, 12, 15, 18, 20]
for i in range(5):
    print('========== max_depth: %d ==========' % depths[i])
    
    # Set up parameters for xgboost
    param = params = {
            'eta': 0.1,
            'max_depth': depths[i],
            'objective': 'multi:softprob',
            'eval_metric': 'mlogloss',
            'num_class': 9,
            'seed': 1,
            'silent': True,
            'nthread' :4
            }

    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]

    model = xgb.train(params, xgb.DMatrix(x1, y1), 500,  watchlist, verbose_eval=250, early_stopping_rounds=100)

========== max_depth: 10 ==========
[0]	train-mlogloss:2.01589	valid-mlogloss:2.04873
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
Stopping. Best iteration:
[123]	train-mlogloss:0.411008	valid-mlogloss:0.944508

========== max_depth: 12 ==========
[0]	train-mlogloss:2.00688	valid-mlogloss:2.04385
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
Stopping. Best iteration:
[113]	train-mlogloss:0.366464	valid-mlogloss:0.947517

========== max_depth: 15 ==========
[0]	train-mlogloss:1.99741	valid-mlogloss:2.03932
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
Stopping. Best iteration:
[79]	train-mlogloss:0.365388	valid-mlogloss:0.963744

========== max_depth: 18 ==========
[0

Play with min_child_weight

In [69]:
%%time
# Adjust the class labels so it starts with 0
y_adj = y-1

# Split into test and train
x1, x2, y1, y2 = train_test_split(X, y_adj, test_size=0.15)

min_child_weights =[1, 2, 5]
for i in range(len(min_child_weights)):
    print('========== min_child_weight: %d ==========' % min_child_weights[i])
    
    # Set up parameters for xgboost
    param = params = {
            'eta': 0.1,
            'max_depth': 15,
            'min_child_weight': min_child_weights[i],
            'objective': 'multi:softprob',
            'eval_metric': 'mlogloss',
            'num_class': 9,
            'seed': 1,
            'silent': True,
            'nthread' :4
            }

    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]

    model = xgb.train(params, xgb.DMatrix(x1, y1), 500,  watchlist, verbose_eval=250, early_stopping_rounds=100)

========== min_child_weight: 1 ==========
[0]	train-mlogloss:2.00035	valid-mlogloss:2.02778
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
Stopping. Best iteration:
[55]	train-mlogloss:0.43163	valid-mlogloss:1.01864

========== min_child_weight: 2 ==========
[0]	train-mlogloss:2.01333	valid-mlogloss:2.03051
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
Stopping. Best iteration:
[60]	train-mlogloss:0.462865	valid-mlogloss:1.01382

========== min_child_weight: 5 ==========
[0]	train-mlogloss:2.04257	valid-mlogloss:2.05592
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
Stopping. Best iteration:
[69]	train-mlogloss:0.52694	valid-mlogloss:1.03538

CPU times: user 13min, sys:

Try different gammas, also make max_delta_step=1

In [72]:
%%time
# Adjust the class labels so it starts with 0
y_adj = y-1

# Split into test and train
x1, x2, y1, y2 = train_test_split(X, y_adj, test_size=0.15)

gammas =[0.6, 0.8, 1]
for i in range(len(gammas)):
    print('========== gamma: %f ==========' % gammas[i])
    
    # Set up parameters for xgboost
    param = params = {
            'eta': 0.1,
            'gamma': gammas[i],
            'max_delta_step': 1,
            'max_depth': 15,
            'min_child_weight': 1,
            'objective': 'multi:softprob',
            'eval_metric': 'mlogloss',
            'num_class': 9,
            'seed': 1,
            'silent': True,
            'nthread' :4
            }

    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]

    model = xgb.train(params, xgb.DMatrix(x1, y1), 500,  watchlist, verbose_eval=250, early_stopping_rounds=100)

========== gamma: 0.600000 ==========
[0]	train-mlogloss:2.09678	valid-mlogloss:2.11528
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
Stopping. Best iteration:
[98]	train-mlogloss:0.405016	valid-mlogloss:1.04103

========== gamma: 0.800000 ==========
[0]	train-mlogloss:2.09735	valid-mlogloss:2.11554
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
Stopping. Best iteration:
[145]	train-mlogloss:0.454069	valid-mlogloss:1.0462

========== gamma: 1.000000 ==========
[0]	train-mlogloss:2.09744	valid-mlogloss:2.11533
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[250]	train-mlogloss:0.489357	valid-mlogloss:1.06041
Stopping. Best iteration:
[229]	train-mlogloss:0.489357	valid-m

Cross Validate with multiple rounds

In [77]:
%%time
# Adjust the class labels so it starts with 0
y_adj = y-1

scores = []

iteration = 5
for i in range(iteration):
    print('========== Iteration %d/%d ==========' % (i, iteration))
    
    # Split into test and train randomly, every iteration
    x1, x2, y1, y2 = train_test_split(X, y_adj, test_size=0.15)
    
    # Set up parameters for xgboost
    param = params = {
            'eta': 0.1,
            'gamma': 0.8,
            'max_delta_step': 1,
            'max_depth': 15,
            'min_child_weight': 1,
            'objective': 'multi:softprob',
            'eval_metric': 'mlogloss',
            'num_class': 9,
            'seed': 1,
            'silent': True,
            'nthread' :4
            }

    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]

    model = xgb.train(params, xgb.DMatrix(x1, y1), 500,  watchlist, 
                      verbose_eval=500, early_stopping_rounds=100)

    score = log_loss(y2, model.predict(xgb.DMatrix(x2), 
                     ntree_limit=model.best_ntree_limit), 
                     labels = list(range(9)))
    scores.append(score)
    print(score)
          
print("Average mlogloss: %f" % np.mean(scores))

========== Iteration 0/5 ==========
[0]	train-mlogloss:2.09675	valid-mlogloss:2.11377
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
Stopping. Best iteration:
[259]	train-mlogloss:0.43171	valid-mlogloss:0.990014

0.990014515522
========== Iteration 1/5 ==========
[0]	train-mlogloss:2.09706	valid-mlogloss:2.11376
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
Stopping. Best iteration:
[93]	train-mlogloss:0.440223	valid-mlogloss:1.03176

1.0317560281
========== Iteration 2/5 ==========
[0]	train-mlogloss:2.09615	valid-mlogloss:2.11687
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
Stopping. Best iteration:
[91]	train-mlogloss:0.457849	valid-mlogloss:1.05146

1.05145463818
